In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

spark = SparkSession.builder.appName("ProductsAndCategories").getOrCreate()

# ID продуктов
productsData = [
    (1, "Хлеб"),
    (2, "Молоко"),
    (3, "Масло"),
    (4, "Сыр"),
    (5, "Яблоки"),
    (6, "Бананы"),
    (7, "Апельсины"),
    (8, "Помидоры"),
    (9, "Огурцы"),
    (10, "Курица"),
    (11, "Говядина"),
    (12, "Рыба"),
    (13, "Рис"),
    (14, "Макароны"),
    (15, "Йогурт"),
    (16, "Кофе"),
    (17, "Сок")
]

# ID категорий
categoriesData = [
    (1, "Хлебобулочные изделия"),
    (2, "Молочные продукты"),
    (3, "Фрукты"),
    (4, "Овощи"),
    (5, "Мясо"),
    (6, "Зерновые"),
    (7, "Напитки"),
    (8, "Готовая еда"),
    (9, "Здоровая еда"),
]

# Связи: продукт - категория
productCategoryData = [
    (1, 1),
    (2, 2),
    (3, 2),
    (4, 2),
    (5, 3),
    (6, 3),
    (7, 3),
    (8, 4),
    (9, 4),
    (9, 9),
    (10, 5),
    (11, 5),
    (12, 5),
    (13, 6),
    (14, 6),
    (15, 2),
    (17, 3),
    (17, 7)
]

# DataFrame для продуктов
productsDataFrame = spark.createDataFrame(productsData, ["product_id", "product_name"])

# DataFrame для категорий
categoriesDataFrame = spark.createDataFrame(categoriesData, ["category_id", "category_name"])

# DataFrame для связей
productCategoryDataFrame = spark.createDataFrame(productCategoryData, ["product_id", "category_id"])

# Получение пар "Имя продукта – Имя категории" 
productCategoryJoin = productCategoryDataFrame.join(productsDataFrame, "product_id").join(categoriesDataFrame, "category_id")
productCategoryPairs = productCategoryJoin.select("product_name", "category_name")

# Поиск продуктов, которые не имеют категорий
productsWithoutCategories = productsDataFrame.join(productCategoryDataFrame, "product_id", "left_anti")
productsWithoutCategories = productsWithoutCategories.select("product_name").withColumn("category_name", lit("Отсутствует категория"))

# Поиск категорий, которые не имеют продуктов
categoriesWithoutProducts = categoriesDataFrame.join(productCategoryDataFrame, "category_id", "left_anti")
categoriesWithoutProducts = categoriesWithoutProducts.select("category_name").withColumn("product_name", lit("Отсутствует продукт"))

# Объединение результатов
finalResult = (
    productCategoryPairs
    .union(productsWithoutCategories)
    .union(categoriesWithoutProducts)
)

# Финальный результат
finalResult.show()


+------------+--------------------+
|product_name|       category_name|
+------------+--------------------+
|        Хлеб|Хлебобулочные изд...|
|      Йогурт|   Молочные продукты|
|         Сыр|   Молочные продукты|
|       Масло|   Молочные продукты|
|      Молоко|   Молочные продукты|
|         Сок|              Фрукты|
|   Апельсины|              Фрукты|
|      Бананы|              Фрукты|
|      Яблоки|              Фрукты|
|      Огурцы|               Овощи|
|    Помидоры|               Овощи|
|        Рыба|                Мясо|
|    Говядина|                Мясо|
|      Курица|                Мясо|
|    Макароны|            Зерновые|
|         Рис|            Зерновые|
|         Сок|             Напитки|
|      Огурцы|        Здоровая еда|
|        Кофе|Отсутствует катег...|
| Готовая еда| Отсутствует продукт|
+------------+--------------------+

